In [ ]:
### Desafio

- Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
    - Dólar
    - Euro
    - Ouro
- Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.
- Base de Dados: [https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing](https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing)
- Para isso, vamos criar uma automação web:
    - Usaremos o selenium (controla o navegador)
    - Importante: baixar o webdriver

In [14]:
# Baixar webdriver 
# Chrome → chromedrive

!pip install selenium


In [16]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# a biblioteca é selenium e um livro é o webdriver
# não precisa importar tudo

# por que não usar pyautogui?

# pyautogui só funciona pro nosso computador
# selenium só controla navegador, não controla um excel instalador por exemplo



In [38]:
# Passo 1: Pegar a cotação do dólar

navegador = webdriver.Chrome("chromedriver.exe")
# navegador = webdriver.Chrome()
# navegador = webdriver.Chrome("chromedriver.exe") 
# o último é para quando o chromedriver estiver no mesmo local do arquivo e nao na mesma pasta do pasta

# Navegador precisa entrar no google

navegador.get("https://www.google.com/")


# encontrar o elemento:
# ir no google, clicar com o botão dreito em cima da pesquisa do google
# clica em 'inspecionar'
# cada local tem seu próprio lugar
# selecionar o XPATH
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
# send_keys - "enviar teclas" -> escrever

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 

print(cotacao_dolar)

C:\Users\bruno\AppData\Local\Temp/ipykernel_4904/579844759.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("chromedriver.exe")


5.5346


In [39]:
# Passo 2: Pegar a cotação do Euro

navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

6.333132088


In [40]:
# Passo 3: Pegar a cotação do Ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)

navegador.quit() # para fechar

324.80


In [41]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [42]:
# Passo 5: Recalcular o preço de cada produto

# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
# tabela.loc[linha,coluna]
# tabela.loc[numero da linha, coluna x]
# ou tabela['nome da linha' 
# == igual
# quero que na coluna moeda, todos os lugares que estejam igual a x, seja convetido em

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)
# float: número em casa decimal

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.534600,5534.544654,1.40,7748.362516
1,Carro Renault,4500.00,Euro,6.333132,28499.094396,2.00,56998.188792
2,Notebook Dell,899.99,Dólar,5.534600,4981.084654,1.70,8467.843912
3,IPhone,799.00,Dólar,5.534600,4422.145400,1.70,7517.647180
4,Carro Fiat,3000.00,Euro,6.333132,18999.396264,1.90,36098.852902
5,Celular Xiaomi,480.48,Dólar,5.534600,2659.264608,2.00,5318.529216
6,Joia 20g,20.00,Ouro,324.800000,6496.000000,1.15,7470.400000


In [43]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)